# data to xlsx func

目前 QS、THE 所用之 json 大同小异，吾亦将 ARWU 数据爬取为彼样 json，故使用统一函数处理之。json 如：

```json
{"data": [{"name": "TUH", "rank": "0", "score": "10422"}]}
```

In [43]:
# a common function to process json gotten by crawler
import requests
import json
import xlsxwriter as xlwt
from bs4 import BeautifulSoup

def ranker_to_col_names(ranker):
    col_name_dic = {'name': 'name', 'rank': 'rank', 'score': 'score'} # , 'region': 'region', 'aliases': 'aliases'
    if ranker == 'QS':
        col_name_dic['name'] = 'title'
        col_name_dic['rank'] = 'rank_display'
        col_name_dic['score'] = 'score'
        col_name_dic['region'] = 'region'
    elif ranker == 'THE':
        col_name_dic['name'] = 'name'
        col_name_dic['rank'] = 'rank'
        col_name_dic['score'] = 'scores_overall'
        col_name_dic['region'] = 'location'
        col_name_dic['aliases'] = 'aliases'
    elif ranker == 'ARWU':
        col_name_dic = col_name_dic
    return col_name_dic

# 将字典数据转换为数组
def parser_page(json, ranker):
    col_name_dic = ranker_to_col_names(ranker)
    if json:
        items = json.get('data')
        for i in range(len(items)):
            item = items[i]
            res = {}
            for col in col_name_dic.keys():
                if col == 'name':
                    res['name'] = BeautifulSoup(item[col_name_dic['name']]).get_text()
                elif col == 'rank':
                    if "=" in item[col_name_dic['rank']]:
                        rk_str = str(item[col_name_dic['rank']]).split('=')[-1]
                        res['rank'] = rk_str
                    else:
                        res['rank'] = item[col_name_dic['rank']]
                else:
                    res[col] = item[col_name_dic[col]]
            yield res

def load_data_to_xlsx(ranker, years):
    workbook = xlwt.Workbook('{0}.xlsx'.format(ranker))
    worksheet_main = workbook.add_worksheet('main')

    col_name_dic = ranker_to_col_names(ranker)

    # 写每一年的分表
    for year in years:
        with open('{0}-json\\{1}.json'.format(ranker, year), 'r', encoding='utf-8') as f:
            data = json.loads(f.read())
        results = parser_page(data, ranker)

        worksheet = workbook.add_worksheet(year)

        headings = list(col_name_dic.keys())
        col = 0
        for heading in headings:
            worksheet.write(0, col, heading)
            col = col+1

        row = 1
        for result in results:
            col = 0
            for k, v in result.items():
                worksheet.write(row, col, v)
                col = col + 1
            row = row + 1

    # 写 main 表
    with open('{0}-json\\{1}.json'.format(ranker, years[0]), 'r', encoding='utf-8') as f:
        data = json.loads(f.read())
    results = parser_page(data, ranker)
    headings_main = ['University Name'] + years
    col = 0
    for heading in headings_main:
        worksheet_main.write(0, col, heading)
        col += 1
    row = 1
    for result in results:
        worksheet_main.write(row, 0, result['name'])
        for year in years:
            worksheet_main.write(
                row, headings_main.index(year), "=VLOOKUP(A{0},'{1}'!A:B,2,0)".format(row + 1, year))
        row += 1

    worksheet_main.set_column('A:A',20) 
    workbook.close()

# QS to grab
using XHR gotten in urls like https://www.topuniversities.com/university-rankings/world-university-rankings/2022

In [5]:
# 从network的xhr文件中发现json文件的url
urls={
    '2022': 'https://www.topuniversities.com/sites/default/files/qs-rankings-data/en/3740566.txt?1623183713?v=1623251204159',
    '2021': 'https://www.topuniversities.com/sites/default/files/qs-rankings-data/en/2057712.txt?1623183364?v=1623254149518',
    '2020': 'https://www.topuniversities.com/sites/default/files/qs-rankings-data/en/914824.txt?1623183839?v=1623254203592',
    '2019': 'https://www.topuniversities.com/sites/default/files/qs-rankings-data/en/397863.txt?1623183911?v=1623254242977',
    '2018': 'https://www.topuniversities.com/sites/default/files/qs-rankings-data/en/357051.txt?1623189642?v=1623254394658',
    '2017': 'https://www.topuniversities.com/sites/default/files/qs-rankings-data/en/357051.txt?1623189642?v=1623254422154',
    '2016': 'https://www.topuniversities.com/sites/default/files/qs-rankings-data/en/326584.txt?1623203022?v=1623254459114',
    '2015': 'https://www.topuniversities.com/sites/default/files/qs-rankings-data/en/299926.txt?1623193600?v=1623254527088'
}

In [ ]:
# crawl
import requests
import json
for year, url in urls.items():
    cont = requests.get(url)
    data = cont.json()   # json数据类型为dict
    with open(r'.\QS-json\{0}.json'.format(year), 'w', encoding='utf-8') as f:
        f.write(json.dumps(data))

# QS to analyse
process data collected from qs website

In [46]:
load_data_to_xlsx('QS', [str(i) for i in range(2022, 2015 - 1, -1)])

# THE data getting
like https://www.timeshighereducation.com/world-university-rankings/2021/world-ranking
THE 的网页和QS一样都把数据存在 XHR 里，所以找到对应的 json 文件链接便可获取完整数据。但要额外注意的是 THE 的网站对 headers 有检查，不加 headers 会返回 403。

In [5]:
# 从network的xhr文件中发现json文件的url
urls={
    '2021': 'https://www.timeshighereducation.com/sites/default/files/the_data_rankings/world_university_rankings_2021_0__fa224219a267a5b9c4287386a97c70ea.json',
    '2020': 'https://www.timeshighereducation.com/sites/default/files/the_data_rankings/world_university_rankings_2020_0__24cc3874b05eea134ee2716dbf93f11a.json',
    '2019': 'https://www.timeshighereducation.com/sites/default/files/the_data_rankings/world_university_rankings_2019_0__8923a34186e552aa8aec863e45bc02d5.json',
    '2018': 'https://www.timeshighereducation.com/sites/default/files/the_data_rankings/world_university_rankings_2018_0__e814f039fcc8ddc45dc6085e4a8a8b66.json',
    '2017': 'https://www.timeshighereducation.com/sites/default/files/the_data_rankings/world_university_rankings_2017_0__06ec07a66faf58bb6171791e5852fe1c.json',
    '2016': 'https://www.timeshighereducation.com/sites/default/files/the_data_rankings/world_university_rankings_2016_0__db6aad7f77c771c83817ced2b2ed3722.json',
    '2015': 'https://www.timeshighereducation.com/sites/default/files/the_data_rankings/world_university_rankings_2015_0__0ea44da985e26d09045368c98f819b91.json'
}
latest_year = list(urls.keys())[0]

In [4]:
# crawl
import requests
import json
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36'}
for year, url in urls.items():
    cont = requests.get(url, headers=headers)
    print(cont)
    data = cont.json()   # json数据类型为dict
    with open(r'.\THE-json\{0}.json'.format(year), 'w', encoding='utf-8') as f:
        f.write(json.dumps(data))

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>


# THE to analyse

In [45]:
load_data_to_xlsx('THE', [str(i) for i in range(2021, 2015 - 1, -1)])

# ARWU data getting

it appears to use js to display data, not XHR. so I use selenium to click and get data per page.

**TODO: solve it that some univs in this ranking has different names in different years, e. g. ETH, ICL, Berkeley**

In [29]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select

option = Options()
option.add_argument("--disable-notifications")

# find limited page number
def get_final_page_number(url):
    url = url
    chrome = webdriver.Chrome('./chromedriver', options=option)
    chrome.get(url)

    soup = BeautifulSoup(chrome.page_source, 'lxml')
    flag = False
    tags_a = soup.find_all("a")
    limited_page = 0
    for tag in tags_a:
        if(tag.text == "•••" and flag == False):
            flag = True
        elif(flag == True):
            try:
                limited_page = int(tag.text)
            except:
                print("ERROR: Error with convert page number to integer.")
            break
    print("limit page:", limited_page)
    chrome.close()
    return limited_page

# get one page's infomation
def get_page_info(soup):
    items = soup.find_all("tr")
    flag = False
    data = []
    cols = ['rank', 'score', 'name']

    for row in items:
        if(flag == False):
            flag = True
            continue
        single_data = {}
        # score and ranking
        try:
            text = row.contents[0].text.lstrip().rstrip()
            single_data['rank'] = text
            text = row.contents[4].text.lstrip().rstrip()
            single_data['score'] = text
        except:
            print("ERROR: score ranking error.")
            pass
        # institution name
        try:
            text = row.find(class_="univ-name").text.lstrip().rstrip()
            single_data['name'] = text
        except:
            try:
                text = row.find(class_="univ-name-normal").text.lstrip().rstrip()
                single_data['name'] = text
            except:
                print("ERROR: university name error.")
                pass
        
        if(len(single_data) == 3):
            data.append(single_data)
        else:
            print(single_data, " row error.")

    return data

In [30]:
# crawler
for year in range(2010, 2020 + 1):
    url = "http://www.shanghairanking.com/rankings/arwu/{}".format(year)
    chrome = webdriver.Chrome('./chromedriver', options=option)
    chrome.get(url)

    count = 0
    limited_page = get_final_page_number(url)
    data = {'data': []}

    while(count < limited_page):
        time.sleep(1)
        print("[Page: %2d] " % (count + 1), end="")

        soup = BeautifulSoup(chrome.page_source, 'lxml')
        data['data'] += get_page_info(soup)
        # start: construct all lists' info
        
        time.sleep(2)

        # click next page
        btns_next_page = chrome.find_elements_by_class_name("ant-pagination-item-link", )
        btns_next_page[-1].click()
        count += 1

    chrome.close()
    print("- finish -")
    with open(r'.\ARWU-json\{0}.json'.format(year), 'w', encoding='utf-8') as f:
        f.write(json.dumps(data))

limit page: 17
[Page:  1] [Page:  2] [Page:  3] [Page:  4] [Page:  5] [Page:  6] [Page:  7] [Page:  8] [Page:  9] [Page: 10] [Page: 11] [Page: 12] [Page: 13] [Page: 14] [Page: 15] [Page: 16] [Page: 17] - finish -
limit page: 17
[Page:  1] [Page:  2] [Page:  3] [Page:  4] [Page:  5] [Page:  6] [Page:  7] [Page:  8] [Page:  9] [Page: 10] [Page: 11] [Page: 12] [Page: 13] [Page: 14] [Page: 15] [Page: 16] [Page: 17] - finish -
limit page: 17
[Page:  1] [Page:  2] [Page:  3] [Page:  4] [Page:  5] [Page:  6] [Page:  7] [Page:  8] [Page:  9] [Page: 10] [Page: 11] [Page: 12] [Page: 13] [Page: 14] [Page: 15] [Page: 16] [Page: 17] - finish -
limit page: 17
[Page:  1] [Page:  2] [Page:  3] [Page:  4] [Page:  5] [Page:  6] [Page:  7] [Page:  8] [Page:  9] [Page: 10] [Page: 11] [Page: 12] [Page: 13] [Page: 14] [Page: 15] [Page: 16] [Page: 17] - finish -
limit page: 17
[Page:  1] [Page:  2] [Page:  3] [Page:  4] [Page:  5] [Page:  6] [Page:  7] [Page:  8] [Page:  9] [Page: 10] [Page: 11] [Page: 12] [

In [44]:
load_data_to_xlsx('ARWU', [str(i) for i in range(2020, 2010 - 1, -1)])